In [1]:
import pandas as pd
from datetime import datetime
from clickhouse_driver import Client
from faker import Faker
import random
import logging


fake = Faker('en-us')

city_zip_mapping = {
    "New York": "10001",
    "Los Angeles": "90001",
    "Chicago": "60601",
    "Houston": "77001",
    "Phoenix": "85001",
    "Philadelphia": "19101",
    "San Antonio": "78201",
    "San Diego": "92101",
    "Dallas": "75201",
    "San Jose": "95101",
    "Austin": "73301",
    "Jacksonville": "32099",
    "San Francisco": "94101",
    "Indianapolis": "46201",
    "Columbus": "43085",
    "Fort Worth": "76101",
    "Charlotte": "28201",
    "Seattle": "98101",
    "Denver": "80201",
    "Washington": "20001",
    "Boston": "02101",
    "El Paso": "79901",
    "Nashville": "37201",
    "Detroit": "48201",
    "Oklahoma City": "73101",
    "Portland": "97201",
    "Las Vegas": "89101",
    "Memphis": "38101",
    "Louisville": "40201",
    "Baltimore": "21201",
    "Milwaukee": "53201",
    "Albuquerque": "87101",
    "Tucson": "85701",
    "Fresno": "93701",
    "Mesa": "85201",
    "Sacramento": "94203",
    "Atlanta": "30301",
    "Kansas City": "64101",
    "Colorado Springs": "80901",
    "Raleigh": "27601",
    "Omaha": "68101",
    "Miami": "33101",
    "Long Beach": "90801",
    "Virginia Beach": "23450",
    "Oakland": "94601",
    "Minneapolis": "55401",
    "Tampa": "33601",
    "Arlington": "22201",
    "New Orleans": "70112",
}

def read_clickhouse_to_dataframe(host,port,user,password,database):
    query = 'SELECT email_address,first_name,last_name,address_1,address_2,gender,postal_code,city,primary_phone_nbr,runid,source_name,file_name FROM registria'
    # Connect to ClickHouse server
    client1 = Client(host=host, port=port, user=user, password=password, database=database)
    try:
        # Execute SQL query to fetch data
        result = client1.execute(query)
        columns = ['email_address', 'first_name', 'last_name', 'address_1', 'address_2', 'gender', 'postal_code', 'city', 'primary_phone_nbr', 'runid', 'source_name', 'file_name']
        df = pd.DataFrame(result, columns=columns)
        df=df.to_dict(orient="records")

    finally:
        # Close ClickHouse connection
        client1.disconnect()

    return df

def generate_us_state_city_data():
    us_state_city_data = []
    state_name = fake.state()
    city_name = random.choice(list(city_zip_mapping.keys()))  # Select a random city from the keys of the dictionary
    zip_code = city_zip_mapping[city_name]  # Get the zip code for the selected city
    address = fake.street_address()
    city_data = {"city": city_name, "zip_code": zip_code, "address": address}
    us_state_city_data.append({"state": state_name, "city": city_data})
    return us_state_city_data

def func_data_mod(df1):
    for val in df1:
        try:
            var1=generate_us_state_city_data()
            val['customer_email']=f"{val['customer_name'].split()[0]}{val['customer_name'].split()[1]}@{fake.free_email_domain()}"
            val['billto_street_1']=fake.street_address()
            val['billto_street_2']=""
            val['billto_city']=var1[0]['city']['city']
            val['billto_zip_code']=var1[0]['city']['zip_code']
            val['shipto_email']=f"{val['shipto_name'].split()[0]}{val['shipto_name'].split()[1]}@{fake.free_email_domain()}"
            val['shipto_street_1']=fake.street_address()
            val['shipto_street_2']=""
            var2=generate_us_state_city_data()
            val['shipto_city']=var2[0]['city']['city']
            val['shipto_zip_code']=var2[0]['city']['zip_code']
            val['insertion_date']=str(datetime.now().date())
        except Exception as e:
            logging.info(e)
    return df1

def main_new(df1,host,port,user,password,database):
    dfs=func_data_mod(df1)
    lis=[]
    df = read_clickhouse_to_dataframe(host,port,user,password,database)
    # df=df[60:100]
    start_percentage = 0
    end_percentage = 60
    total_rows = len(df)
    start_index = int(total_rows * (start_percentage / 100))
    end_index = int(total_rows * (end_percentage / 100))
    df = df[start_index:end_index]
    for i in range(len(df)):
        try:
            dfs[i]['customer_name']=f"{df[i]['last_name']} {df[i]['first_name']}"
            dfs[i]['customer_email']=df[i]['email_address']
            dfs[i]['file_name']=df[i]['file_name']
            dfs[i]['source']=df[i]['source_name']
            dfs[i]['runid']=df[i]['runid']
            lis.append(dfs[i])
        except Exception as e:
            logging.info(e)
    for j in range(len(df), len(dfs)):
        try:
            lis.append(dfs[j])
        except Exception as e:
            logging.info(e)
    return lis